In [4]:
import cv2
import dlib
import numpy as np

# Load the Dlib face detector and landmark predictor
# The `shape_predictor_68_face_landmarks.dat` file must be in the same directory.
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("fld311/shape_predictor_68_face_landmarks.dat")

# Open the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

print("Webcam successfully opened. Press 'q' to quit.")

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert the frame to grayscale for faster processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    # The `1` means we are upscaling the image 1 time to find more faces.
    faces = face_detector(gray, 1)

    # Loop through all detected faces
    for face in faces:
        # Predict the 68 facial landmarks for the current face
        landmarks = landmark_predictor(gray, face)

        # Draw a rectangle around the face
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Convert the landmarks to a NumPy array for easier drawing
        points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)])

        # Draw each of the 68 landmarks
        for (x, y) in points:
            cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)

    # Display the resulting frame with landmarks
    cv2.imshow('Facial Landmark Detection', frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and destroy all windows
cap.release()
cv2.destroyAllWindows()

Webcam successfully opened. Press 'q' to quit.


# An attempt to make plugin for Blender

In [6]:
import cv2
import dlib
import numpy as np
import socket
import json
import time

# Load the Dlib face detector and landmark predictor
# The `shape_predictor_68_face_landmarks.dat` file must be in the same directory.
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Socket configuration
HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)
sock = None

# Open the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

print("Webcam successfully opened.")
print(f"Press 'r' to connect to Blender and stream data to {HOST}:{PORT}.")
print("Press 'q' to quit.")

is_connected = False
connect_attempted = False

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert the frame to grayscale for faster processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_detector(gray, 1)

    # Loop through all detected faces
    for face in faces:
        # Predict the 68 facial landmarks for the current face
        landmarks = landmark_predictor(gray, face)

        # Draw a rectangle around the face
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Convert the landmarks to a list of lists for JSON serialization
        points = [[landmarks.part(i).x, landmarks.part(i).y] for i in range(68)]

        if is_connected:
            try:
                # Serialize the points and send them over the socket
                json_data = json.dumps(points)
                # We need to send the data size first so the receiver knows how much to read
                message = f"{len(json_data):<10}".encode('utf-8') + json_data.encode('utf-8')
                sock.sendall(message)
                cv2.putText(frame, "STREAMING...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            except (socket.error, ConnectionResetError) as e:
                print(f"Connection error: {e}")
                is_connected = False
                sock.close()
                sock = None
                print("Connection to Blender lost. Press 'r' to reconnect.")

        # Draw each of the 68 landmarks
        for (x, y) in points:
            cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)

    # Display the resulting frame with landmarks
    cv2.imshow('Facial Landmark Detection', frame)

    # Handle key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('r'):
        if not is_connected and not connect_attempted:
            print("Connecting to Blender...")
            connect_attempted = True
            try:
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.connect((HOST, PORT))
                sock.settimeout(0.1) # Set a timeout to prevent blocking
                is_connected = True
                print("Connection established! Streaming data to Blender.")
            except socket.error as e:
                print(f"Failed to connect to Blender: {e}")
                print("Please ensure the Blender plugin is running and the server is listening.")
                sock = None
        elif is_connected:
            print("Disconnecting from Blender.")
            is_connected = False
            if sock:
                sock.close()
                sock = None
    elif key == ord('q'):
        break

# Clean up
if sock:
    sock.close()
cap.release()
cv2.destroyAllWindows()


Webcam successfully opened.
Press 'r' to connect to Blender and stream data to 127.0.0.1:65432.
Press 'q' to quit.
Connecting to Blender...
Connection established! Streaming data to Blender.
Disconnecting from Blender.
